# Spatial Integration of Data

In [3]:
import pandas as pd
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from esda import G_Local
from sklearn.metrics import pairwise_distances
import tifffile
import cv2
import libpysal
from mpl_toolkits.axes_grid1 import AxesGrid
from skimage.transform import probabilistic_hough_line
from skimage.draw import line as draw_line
from skimage.morphology import binary_dilation, binary_opening, disk
import skimage.morphology as morph
from scipy.ndimage import label as scilabel
from sklearn.neighbors import NearestNeighbors
import warnings, tqdm

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

## Load Metals Data

In [ ]:
with open("./metals_df.pkl", "rb") as fh:
    metal_abundances = pd.DataFrame.from_records(p.load(fh)).drop(columns=["index"])

## Load Gene Counts Data (Spatial Transcriptomics)

In [ ]:
with open("./gene_counts.pkl", "rb") as fh:
    gene_counts = pd.DataFrame.from_records(p.load(fh)).drop(columns=["index"])

## Compute Pairwise Distances for Metal Abundance Coordinates

In [ ]:
s = pd.Series(pairwise_distances(metal_abundances[["ST_x", "ST_y"]]).flatten())

## Load H&E Image

In [ ]:
hne_image = tifffile.imread('./_SS12251_092842.svs')
im_small = cv2.resize(hne_image, None, fx=1/15, fy=1/15)

## Define Function to Shift Colormap

In [ ]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    """
    Function to offset the center of a colormap, useful for visualizing data where the midpoint is critical.
    """
    cdict = {'red': [], 'green': [], 'blue': [], 'alpha': []}
    reg_index = np.linspace(start, stop, 257)
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False),
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])
    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)
        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))
    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)
    return newcmap

## Perform Local G Analysis for Metal Hotspots

### Set threshold and compute spatial weights

In [ ]:
threshold_mult = 1
element = "Fe56"
w = libpysal.weights.DistanceBand(metal_abundances[["ST_x", "ST_y"]], threshold=(s[s>0].min()*np.sqrt(2))*threshold_mult+5)

### Compute Local G statistic

In [ ]:
lg = G_Local(metal_abundances[element], w, star=True)
vmax, vmin = lg.Zs.max(), lg.Zs.min()

## Visualize Metal Hotspots on H&E Image

In [ ]:
plt.imshow(im_small)
plt.scatter(*(metal_abundances[["ST_x", "ST_y"]].values / 15).T.tolist(),
            c=lg.Zs, cmap=shiftedColorMap(matplotlib.colormaps["seismic"], midpoint=1 - vmax/(vmax + abs(vmin))), s=0.25)
plt.xlabel("x coord")
plt.ylabel("y coord")
plt.colorbar(label=f"{element} HotSpot")
plt.show()

## Load and Process Metal Data for Spatial Alignment

In [ ]:
# Pre-made stack of metals data
xarray_metals = pd.read_pickle("./stack.pkl")
ref_img = cv2.imread("H_E_resized.tiff")
homography_matrix = pd.read_pickle("./homography_matrix_20.pkl")

### Warp metal images to align with H&E image

In [ ]:
# Homography matrix warping
warped_elements = {str(np.array(element)): cv2.warpPerspective(
    (xarray_metals.loc[..., element]).values, homography_matrix,
    (ref_img.shape[1], ref_img.shape[0]), borderValue=-1000)
    for element in xarray_metals.coords["elements"]}

### Replace borderValue (-1000) with NaN

In [ ]:
for k in warped_elements:
    warped_elements[k][warped_elements[k] == -1000] = np.nan

## Save Processed Data

In [ ]:
pd.to_pickle(warped_elements, "./warped_metals.pkl")

## Load ST Data and Align with Metals Data

In [ ]:
with open('./092842_17.pkl', 'rb') as f:
    slide = pickle.load(f)
slide_numpy = pd.read_pickle('./092842_17_np.pkl')
gene_names = slide_numpy['genes']

In [ ]:
ST_HE_coords = np.array([(np.array(slide.spot_locations.image_x[i]),
                          np.array(slide.spot_locations.image_y[i])) for i in range(6159)])

### Nearest Neighbor Matching Between Metal and ST Data

In [ ]:
nn = NearestNeighbors(n_neighbors=1).fit(ST_HE_coords)
distances, indices = nn.kneighbors(metal_abundances[['ST_x', 'ST_y']].values)

### Create DataFrame for Mapped Spots

In [ ]:
ST_spots_df = pd.DataFrame({
    'ST_spot_ID': range(len(ST_HE_coords)),
    'ST_x': ST_HE_coords[:, 0],
    'ST_y': ST_HE_coords[:, 1]
})

In [ ]:
metal_spots_df = pd.DataFrame({
    'metal_spot_ID': range(len(metal_abundances)),
    'ST_spot_ID': indices.flatten()
})

### Merge Metal and ST Data

In [ ]:
metals_data_coords_v2 = metal_abundances.copy()
metals_data_coords_v2['ST_spot_ID'] = metal_spots_df['ST_spot_ID']
metals_data_coords_v2 = metals_data_coords_v2.drop(['ST_x', 'ST_y'], axis=1)

### Group by ST Spot ID and Save

In [ ]:
grouped_metal_spots = metals_data_coords_v2.groupby('ST_spot_ID').mean().reset_index()
grouped_metal_spots = grouped_metal_spots.merge(ST_spots_df, on='ST_spot_ID')
pd.to_pickle(grouped_metal_spots, "./grouped_ST_metal_spots_JL.pkl")

In [ ]:
#Summary
print("Data processing complete. The grouped metal spots dataset has been saved.")